In [11]:
import sys
sys.path.append('..')

import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal

from models.sac import SAC


In [12]:
class Actor(nn.Module):
    def __init__(self, n_sts: int, n_acts: int, dim: int):
        super(Actor, self).__init__()
                
        self.fc1 = nn.Linear(n_sts, dim)
        self.fc_mu = nn.Linear(dim, n_acts)
        self.fc_std = nn.Linear(dim, n_acts)
        
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
    
    def forward(self, x):
        x = self.relu1(self.fc1(x))
        mu = self.fc_mu(x)
        std = F.softplus(self.fc_std(x))
        dist = Normal(mu, std)
        act = dist.rsample()
        log_prob = dist.log_prob(act)
        real_act = torch.tanh(act)
        real_log_prob = log_prob - torch.log(1 - torch.tanh(act).pow(2) + 1e-7)

        return real_act, real_log_prob.sum(-1, keepdims=True)

In [13]:
class QCritic(nn.Module): 
    def __init__(self, n_sts: int, act_dim: int, dim: int):
        super(QCritic, self).__init__()
                
        self.fc1 = nn.Linear(n_sts + act_dim, dim)
        self.fc2 = nn.Linear(dim, 1)
        
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
                
    def forward(self, xs):
        x, a = xs
        
        x = self.relu1(self.fc1(torch.cat([x, a], 1)))
        x = self.fc2(x)
        
        return x

In [14]:
lr = 3e-4
tau = 5e-3
eps = 0
gamma = 0.99

n_sts = 17
n_acts = 6
act_dim = 6

dim = 256

n_epis = 5000
n_epochs = 100
n_rollout = 10

In [15]:
act = Actor(n_sts, n_acts, dim)
act_opt = torch.optim.Adam(act.parameters(), lr=lr)

qcri1 = QCritic(n_sts, act_dim, dim)
qcri1_opt = torch.optim.Adam(qcri1.parameters(), lr=lr)

qcri2 = QCritic(n_sts, act_dim, dim)
qcri2_opt = torch.optim.Adam(qcri2.parameters(), lr=lr)

In [16]:
env = gym.make("HalfCheetah-v4", render_mode='human')
model = SAC(env, n_acts, act, act_opt, qcri1, qcri1_opt, qcri2, qcri2_opt, eps=eps, tau=tau, target_entropy=-n_acts)

In [17]:
model.train(n_epis, n_epochs, n_rollout)

step: 210, score: -5.734494841786448, n_buffer: 210
step: 410, score: -3.1786696235398937, n_buffer: 410
step: 610, score: -1.7912375217086471, n_buffer: 610
step: 810, score: -2.3999593779925954, n_buffer: 810
step: 1210, score: -3.2814117045145665, n_buffer: 1210
step: 1410, score: -3.0395149764052696, n_buffer: 1410
step: 1610, score: -1.2518656347235206, n_buffer: 1610
step: 1810, score: -3.1234966880498636, n_buffer: 1810
step: 2210, score: -3.341076408073923, n_buffer: 2210
step: 2410, score: -0.40365557795966156, n_buffer: 2410
step: 2610, score: -1.557820328561379, n_buffer: 2610
step: 2810, score: -1.420139083834035, n_buffer: 2810
step: 3210, score: -0.3582866202344723, n_buffer: 3210
step: 3410, score: -1.2547449820812078, n_buffer: 3410
step: 3610, score: -0.3315413344076572, n_buffer: 3610
step: 3810, score: -0.32370495059924514, n_buffer: 3810
step: 4210, score: -1.0710117590659773, n_buffer: 4210
step: 4410, score: -5.476672035549774, n_buffer: 4410
step: 4610, score: -3

KeyboardInterrupt: 

In [ ]:
env.close()